# Amazon recommender

## Data import + cleaning

In [1]:
#libraries
import pandas as pd, numpy as np, matplotlib.pyplot as plt, \
seaborn as sns

In [4]:
#fetch data for TRAIN
orders = pd.read_csv('../data/01-Jan-2008_to_18-May-2021.csv')

In [5]:
#peek at data for TRAIN
orders.head()

,Order Date,Order ID,Title,Category,ASIN/ISBN,UNSPSC Code,Website,Release Date,Condition,Seller,...,Carrier Name & Tracking Number,Item Subtotal,Item Subtotal Tax,Item Total,Tax Exemption Applied,Tax Exemption Type,Exemption Opt-Out,Buyer Name,Currency,Group Name
0,12/15/08,002-9208212-4089021,NaN,NaN,B0006HGNJ4,NaN,Amazon.com,NaN,NaN,Watch Values,...,USPS(9101805213907299379333),$24.95,$0.00,$24.95,NaN,NaN,NaN,Veronica,USD,NaN
1,03/23/09,058-3723913-1124538,The Wall,ABIS_MUSIC,B000006TRV,55111512.0,Amazon.com,1994-01-01T00:00,new,megahitrecords,...,NaN,$15.24,$0.00,$15.24,NaN,NaN,NaN,Veronica,USD,NaN
2,01/07/10,105-6872349-1957028,Dreamland,ABIS_MUSIC,B000002VRQ,55111512.0,Amazon.com,1996-07-30T00:00:01,new,Amazon.com,...,USPS(9102001206749319387592),$6.99,$0.66,$7.65,NaN,NaN,NaN,Veronica,USD,NaN
3,06/01/10,102-2903451-7921061,"Neotech Accordion Harness, Original - Padded S...",INSTRUMENT_PARTS_AND_ACCESSORIES,B000E2F3UQ,60131500.0,Amazon.com,NaN,new,Hairy Harry's Music,...,USPS(9101150134711197827330),$38.95,$0.00,$38.95,NaN,NaN,NaN,Veronica,USD,NaN
4,12/19/10,104-0250440-8561850,Vista International 9200S Silver Bubble Umbrella,BAG,B00383QNIO,53102505.0,Amazon.com,NaN,new,Better Canes and Umbrellas Inc.,...,UPS(1Z437Y5W0348615590),$22.00,$0.00,$22.00,NaN,NaN,NaN,Veronica,USD,NaN


In [6]:
#understand prospective features
orders.columns

Index(['Order Date', 'Order ID', 'Title', 'Category', 'ASIN/ISBN',
       'UNSPSC Code', 'Website', 'Release Date', 'Condition', 'Seller',
       'Seller Credentials', 'List Price Per Unit', 'Purchase Price Per Unit',
       'Quantity', 'Payment Instrument Type', 'Purchase Order Number',
       'PO Line Number', 'Ordering Customer Email', 'Shipment Date',
       'Shipping Address Name', 'Shipping Address Street 1',
       'Shipping Address Street 2', 'Shipping Address City',
       'Shipping Address State', 'Shipping Address Zip', 'Order Status',
       'Carrier Name & Tracking Number', 'Item Subtotal', 'Item Subtotal Tax',
       'Item Total', 'Tax Exemption Applied', 'Tax Exemption Type',
       'Exemption Opt-Out', 'Buyer Name', 'Currency', 'Group Name'],
      dtype='object')

In [7]:
#understand size of data
orders.shape

(2242, 36)

In [8]:
#data types we are dealing with and presence of nulls
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2242 entries, 0 to 2241
Data columns (total 36 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Order Date                      2242 non-null   object 
 1   Order ID                        2242 non-null   object 
 2   Title                           2163 non-null   object 
 3   Category                        2163 non-null   object 
 4   ASIN/ISBN                       2242 non-null   object 
 5   UNSPSC Code                     2163 non-null   float64
 6   Website                         2242 non-null   object 
 7   Release Date                    77 non-null     object 
 8   Condition                       2241 non-null   object 
 9   Seller                          2241 non-null   object 
 10  Seller Credentials              5 non-null      object 
 11  List Price Per Unit             2242 non-null   object 
 12  Purchase Price Per Unit         22

In [10]:
#descriptive stats
orders.describe().T

,count,mean,std,min,25%,50%,75%,max
UNSPSC Code,2163.0,4.909079e+07,9.088133e+06,10111300.0,50000000.0,50202300.0,53102500.5,78130000.0
Quantity,2242.0,1.102587e+00,4.331248e-01,0.0,1.0,1.0,1.0,7.0
Purchase Order Number,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PO Line Number,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Shipping Address Street 2,641.0,5.040000e+02,0.000000e+00,504.0,504.0,504.0,504.0,504.0
Tax Exemption Type,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Group Name,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#import data for TEST
#from https://stackoverflow.com/questions/9652832/how-to-load-a-tsv-file-into-a-pandas-dataframe
#read_csv('path_to_file', sep='\t')
products = pd.read_csv(
    '../data/marketing_sample_for_amazon_com-amazon_product_data__20200401_20200630__30k_data.tsv',
sep='\t')

In [7]:
#peek
products.head()

,Uniq Id,Crawl Timestamp,Dataset Origin,Product Id,Product Barcode,Product Company Type Source,Product Brand Source,Product Brand Normalised Source,Product Name Source,Match Rank,...,Product Currency,Product Available Inventory,Product Image Url,Product Model Number,Product Tags,Product Contents,Product Rating,Product Reviews Count,Bsr,Joining Key
0,690e47123b04e15df768a9f54f1654e8,2020-05-13 11:02:04 +0000,NaN,ed177532df52b8b2fdba6cff731b9d00,NaN,Competitor,NaN,NaN,NaN,NaN,...,USD,999999999,https://images-na.ssl-images-amazon.com/images...,NaN,100% Linen Table Runner 14x90 Tailored with Mi...,100% Pure Linen Imported 100% Pure Linen table...,3.5,4,"#83,321 in Kitchen & Dining (See Top 100 in Ki...",78af1f66f1e365647484688360621e5e
1,b02a91cfdae2c5596b568e771b402176,2020-06-27 15:31:29 +0000,NaN,8756f065784790599a218174bc932395,UPC: 799460760240,Competitor,NaN,NaN,NaN,NaN,...,USD,999999999,https://images-na.ssl-images-amazon.com/images...,NL-F-5-2,2 pcs/set LONG Home Sauna Spa Exfoliating Nylo...,The nylon exfoliating cloth is the best bath p...,0.0,0,"#175,862 in Beauty & Personal Care (See Top 10...",83e8d81095fe218d5bb580194669e0c4
2,1cf8726d3917d5dc7c02265a9c296eba,2020-06-06 09:47:03 +0000,NaN,c4fdb379ec7f7cc705dc1bf543376ba1,NaN,Competitor,NaN,NaN,NaN,NaN,...,USD,999999999,https://images-na.ssl-images-amazon.com/images...,NaN,Qsbon 3-Piece Stainless Steel Soup Ladle Spoon...,Make sure this fits by entering your model num...,0.0,0,"#96,860 in Kitchen & Dining (See Top 100 in Ki...",65b1e1586c83306acd22587da09602ff
3,044c2f4f29aba8ca37fa82e3d7b75866,2020-05-18 02:10:59 +0000,NaN,28a30d84bb704fb7870963590b628ae2,NaN,Competitor,NaN,NaN,NaN,NaN,...,USD,999999999,https://images-na.ssl-images-amazon.com/images...,NaN,Lovef 5Pcs Bunny Ear Cosmetic Headband Fashion...,"Material: plush; Soft plush fabric, elastic ha...",0.0,0,"#131,086 in Beauty & Personal Care (See Top 10...",ba26e939fb6f8fb5c789c6f82e84a7b1
4,624ba0f4b9c8d55b8125395c4c31448a,2020-05-28 10:01:53 +0000,NaN,aed911c4f693f4489487a2b462c4134f,NaN,Competitor,NaN,NaN,NaN,NaN,...,USD,999999999,https://images-na.ssl-images-amazon.com/images...,NaN,"LIUMY Ginger Germinal Oil, Hair Growth Serum-G...",►【ANTI-HAIR LOSS PROTECTION】Nourish hair roots...,3.4,30,"#52,837 in Beauty & Personal Care (See Top 100...",0369f977ed425930ff63fe7658a2204f


In [8]:
products.columns

Index(['Uniq Id', 'Crawl Timestamp', 'Dataset Origin', 'Product Id',
       'Product Barcode', 'Product Company Type Source',
       'Product Brand Source', 'Product Brand Normalised Source',
       'Product Name Source', 'Match Rank', 'Match Score', 'Match Type',
       'Retailer', 'Product Category', 'Product Brand', 'Product Name',
       'Product Price', 'Sku', 'Upc', 'Product Url', 'Market',
       'Product Description', 'Product Currency',
       'Product Available Inventory', 'Product Image Url',
       'Product Model Number', 'Product Tags', 'Product Contents',
       'Product Rating', 'Product Reviews Count', 'Bsr', 'Joining Key'],
      dtype='object')

## EDA

## Modeling